# 1. Fourieranalyse lang andauernder Signale
## a) Routine zum Einlesen:
import pyaudio
import numpy
import matplotlib.pyplot as plt
import time
import os
import re

def find_last_numbered_file(directory):
    pattern = r'^audioaufname_(\d+)'  # Muster, um nach Dateien mit dem Namen 'audio_#' zu suchen

    existing_files = [file for file in os.listdir(directory) if re.match(pattern, file)]
    numbers = []

    for file_name in existing_files:
        match = re.match(pattern, file_name)
        if match:
            numbers.append(int(match.group(1)))

    if numbers:
        return max(numbers)
    else:
        return 0  # Wenn keine Dateien gefunden werden, starte bei 0

directory = r'TestdatenJona'  # Hier den Pfad zu deinem Verzeichnis eintragen

last_number = find_last_numbered_file(directory)
next_number = last_number + 1

next_filename = f'TestdatenJona\\audioaufname_{next_number}'
print(f'Der nächste Dateiname sollte {next_filename} sein.')


FORMAT = pyaudio.paInt16
SAMPLEFREQ = 44100
FRAMESIZE = 1024
NOFFRAMES = 220
p = pyaudio.PyAudio()
print('running')
stream = p.open(format=FORMAT,channels=1,rate=SAMPLEFREQ,
input=True,frames_per_buffer=FRAMESIZE)
data = stream.read(NOFFRAMES*FRAMESIZE)
decoded = numpy.frombuffer(data, dtype=int);
stream.stop_stream()
stream.close()
p.terminate()
print('done')
plt.plot(decoded)
plt.ylabel('Frequenz')
plt.xlabel('Zeit')
plt.show()
filename = 'audioaufname' + str(time.time())
numpy.save(next_filename, decoded)

In [1]:
!pip install numpy
import numpy as np
!pip install matplotlib
import matplotlib
import matplotlib.pyplot as plt
import os
import re
import time
import scipy.signal as sgl
import scipy as sp

In [2]:
# d)
%matplotlib inline
def zerteilen(array):
    gauss_window = np.array(sgl.windows.gaussian(512, 512 / 4))
    for i in range(0, len(array) - 511, 256):
        yield np.concatenate([[0] * i, list(gauss_window * array[i:i + 512]), [0] * (len(array) - (i + 512))])

def windowing_func(array):
    window = np.array(list(zerteilen(array)))
    return np.abs(np.fft.fft(window)).mean(0)

In [ ]:
def get_index(data):
    threshold = max(data[:500]) * 3
    i = 0
    for x in data:
        # print(x)
        # print(x > threshold)
        if((x > threshold)):
            return i
        i += 1
    return 0
    

def cut_samples(directory):
    pattern = r'^audioaufname_(\d+)'  # Muster, um nach Dateien mit dem Namen 'audio_#' zu suchen

    existing_files = [file for file in os.listdir(directory) if re.match(pattern, file)]
    numbers = []

    for file_name in existing_files:
        match = re.match(pattern, file_name)
        if match:
            # numbers.append(int(match.group(1)))
            data = np.load(directory + '\\' + file_name)
            one_second = int(data.size / 5)
            # print("second: " + str(one_second))
            data = data[15000:]
            # print(max(data[:50]))
            index = get_index(data)
            # print(index)
            data = data[index:]
            data = data[:one_second]

            plt.plot(data)
            plt.ylabel('Frequenz')
            plt.xlabel('Zeit')
            plt.show()


            # Amplitudenspektrum
            #print(data)
            fourier = np.fft.fft(data)
            # print(fourier)
            
            spektrum = np.abs(fourier)
            #delta_t = 0.000005
            delta_t = np.abs(1 / one_second)
            print("Detla t = ", delta_t)
            
            
            freq = range(0, len(data), 1) / (len(data) * delta_t)
            
            plt.plot(freq, spektrum, 'b')
            plt.title('Amplitudenspektrum')
            plt.ylabel('Amplitude')
            plt.xlabel('Frequenz')
            plt.grid(True)
            plt.show()


            # d) 
            windowed_data = windowing_func(data)
            plt.plot(windowed_data);
            plt.title('Amplitudenspektrum windowed_data')
            plt.ylabel('Amplitude')
            plt.xlabel('Frequenz')
            plt.show()


cut_samples(r'Trainingsdaten')
cut_samples(r'TestdatenTobi')
cut_samples(r'TestdatenJona')